In [1]:
#This script "coregisters" different datasets.
#To adjust this to your needs, you will have to change the hard coded filenames.

#import the libraries we will need.
from Utils_coreg import gridRead, coregRaster, coregPoint, readTopologyPlatepolygonFile
import numpy
import pygplates
from matplotlib import pyplot as plt
import time
import pickle

Imported pygplates.
Imported shapefile.
Imported numpy.
Imported scipy.


In [2]:
#Define two big functions that coregister 
#data in two slightly different ways

In [3]:
def coregLoop(pointlist,ages,plateIDs):
    '''
    coregLoop

    This script reconstructs a shapefile of points to their birth time and 
    coregisters each point with another set of points, or a raster file.

    INPUTS:
    pointlist - list of lat/lon
    ages - list of ages corresponding to lat/lon point
    plateIDs - list of plateIDs corresponding to lat/lon point.

    Hardcoded filenames/variables must be changed below:
    input_rotation_filename - Rotation file 
    rasterfile - Time dependent raster files
    f - Time dependent kinemtic csv outputs from 'convergence.py'

    OUTPUTS:
    Coregistered array: List of lat/lons with properties.
    
    METHOD:
    Takes a set of points 
    Rotates the points back to their birth position
    Determines the point's birth position geophysical properties (coregisters)
    '''
    
    #Set up a list to store the data
    timeSteps=1 # +- Myrs around point to store. Default 1 which means just the measured point age.
    noVariables=18 #The number of variables you save
    Nshp=len(pointlist)

    coregData=numpy.zeros((Nshp,timeSteps,noVariables))

    #Create a rotation model to rotate the points back in time
    input_rotation_filename = "Muller_gplates/Global_EarthByte_230-0Ma_GK07_AREPS.rot"          
    
    file_registry = pygplates.FeatureCollectionFileFormatRegistry()
    rotation_feature_collection = file_registry.read(input_rotation_filename)
    rotation_model = pygplates.RotationModel([rotation_feature_collection])


    #Loop over all the samples, coregistering each one.
    for i, currentPoint in enumerate(pointlist):

        lat=currentPoint[1]
        lon=currentPoint[0]
        age=ages[i]
        plateID=int(plateIDs[i])
        
        print("Deposit:", i, "of", Nshp, "Lat:", lat, "Lon:", lon, "Age:", age,"PlateID:",plateID)

        #Loop through each time step in the plate model
        for time in xrange(0,230,1):
        
            #If the point was formed at the current time (or [timeStepsMyr ] prior/after) then we 
            #want to find the surrounding plate properties.
            #if  (time > (age-10)) and (time < (age+10)) or (time > (age+20)) and (time < (age+30)):
            if  (time > (age-timeSteps)) and (time < (age+timeSteps)):
                t=int(numpy.floor(time-age))
                #print(t,time,age)
                
                #A raster file, to coregister with, these points have already been rotated
                rasterfile="Muller_etal_2016_AREPS_Agegrids_v1.11/netCDF_0-230Ma/EarthByte_AREPS_v1.11_Muller_etal_2016_AgeGrid-"+str(time)+".nc"
                [x,y,z]=gridRead(rasterfile)
    
                #A vector file to coregister with, these points have already been rotated
                f=numpy.loadtxt("Muller_convergence/subStats_"+str(time)+".csv", delimiter=',')  
                lonlat=f[:,0:2]
                
    
                #-------------------#
                #Reconstruct a point to its birth position
                #-------------------#
                latlonPoint = pygplates.LatLonPoint(lat,lon)
                point_to_rotate = pygplates.convert_lat_lon_point_to_point_on_sphere(latlonPoint)
        
                finite_rotation = rotation_model.get_rotation(time, plateID)
                birthPosition = finite_rotation * point_to_rotate
        
                latlonBirth = pygplates.convert_point_on_sphere_to_lat_lon_point(birthPosition)
                #allPoints = finite_rotation * pointSet_to_rotate
        
                latBirth = latlonBirth.get_latitude()
                lonBirth = latlonBirth.get_longitude()
    
                #-------------------#
                #Set the points for coregistering
                region=5.0 #degrees
                
                #-------------------#
                #Coregisterring raster 1
                #-------------------#
                #Find the region in index units
                r=numpy.round(region/(x[1]-x[0]))
    
                #Find the index unit of lat and lon
                idxLon = (numpy.abs(x-lonBirth)).argmin()
                idxLat = (numpy.abs(y-latBirth)).argmin()
                
                #Raster 1
                c2=coregRaster([idxLon,idxLat],z,r)
                #Hack to search further around the age grid if it can't find a match, note index units (not degrees)
                if numpy.isnan(c2):
                    c2=coregRaster([idxLon,idxLat],z,r+150.0)
                    print("Trying raster region: ", r+150.0)

                #-------------------#
                #Coregisterring vector 1
                #-------------------#
                index=coregPoint([lonBirth,latBirth],lonlat,region)
                if index=='inf':
                    print("trying index region", region+15)
                    index=coregPoint([lonBirth,latBirth],lonlat,region+15.0)
    
                if numpy.isnan(c2) or index=='inf':
                    print("Skipping:", i, age, t, time, c2, index, lonBirth,latBirth)
    
                else:
                    #Vector 1
                    segmentLength=f[index,3]
                    slabLength=f[index,9]
                    distSlabEdge=f[index,15]
                    
                    SPcoregNor=f[index,4]
                    SPcoregPar=f[index,12]
                    OPcoregNor=f[index,5]
                    OPcoregPar=f[index,13]
                    CONVcoregNor=f[index,10]
                    CONVcoregPar=f[index,11]
                    
                    subPolCoreg=f[index,8]
                    subOblCoreg=f[index,7]
                    coregData[i,t,:]=[lon,lat,lonBirth,latBirth,age,t,c2,segmentLength,slabLength,distSlabEdge,SPcoregNor,SPcoregPar,OPcoregNor,OPcoregPar,CONVcoregNor,CONVcoregPar,subPolCoreg,subOblCoreg]
    
    #Return the filled coregistered array
    return(coregData)

In [4]:
def coregLoopHistory(pointlist,ts0=0,ts1=230,plateID=201):
    '''
    coregLoopHistory

    This script reconstructs a list of points throughout history and 
    coregisters each point with another set of points, or a raster file.

    INPUTS:
    pointlist - list of lat/lon
    ts0 - time step to rotate points from (probably 0Ma)
    ts1 - time step to rotate points to (probably 230Ma).

    Hardcoded filenames/variables must be changed below:
    input_rotation_filename - Rotation file 
    rasterfile - Time dependent raster files
    f - Time dependent kinemtic csv outputs from 'convergence.py'

    OUTPUTS:
    Coregistered array: List of lat/lons with properties.

    METHOD:
    Takes a set of points 
    Rotates the points back to their birth position
    Determines the point's birth position geophysical properties (coregisters)
    '''
    
    #Set up an array to store the data
    timeSteps=ts1-ts0 #The length of time before mineralisation you care about
    noVariables=18 #The number of variables you save
    Nshp = len(pointlist)
    coregData=numpy.zeros((Nshp,timeSteps,noVariables))
    
    #Create a rotation model to rotate the points back in time
    input_rotation_filename = "Muller_gplates/Global_EarthByte_230-0Ma_GK07_AREPS.rot"

    file_registry = pygplates.FeatureCollectionFileFormatRegistry()
    rotation_feature_collection = file_registry.read(input_rotation_filename)
    rotation_model = pygplates.RotationModel([rotation_feature_collection])


    #Loop through each time step in the plate model
    for time in xrange(ts0,ts1,1):

        #A raster file, to coregister with, these points have already been rotated
        rasterfile="Muller_etal_2016_AREPS_Agegrids_v1.11/netCDF_0-230Ma/EarthByte_AREPS_v1.11_Muller_etal_2016_AgeGrid-"+str(time)+".nc"
        [x,y,z]=gridRead(rasterfile)

        #A vector file to coregister with, these points have already been rotated
        f=numpy.loadtxt("Muller_convergence/subStats_"+str(time)+".csv", delimiter=',')   

        lonlat=f[:,0:2]


        for i, currentPoint in enumerate(pointlist):
                
                shapeArray=currentPoint
                age=ts0

                t=time-age
                print(i, age, time, t, plateID)

                #-------------------#
                #Reconstruct a point to its birth position
                #-------------------#
                latlonPoint = pygplates.LatLonPoint(shapeArray[1],shapeArray[0])
                
                
                point_to_rotate = pygplates.convert_lat_lon_point_to_point_on_sphere(latlonPoint)

                finite_rotation = rotation_model.get_rotation(time, plateID)
                birthPosition = finite_rotation * point_to_rotate

                latlonBirth = pygplates.convert_point_on_sphere_to_lat_lon_point(birthPosition)
                #allPoints = finite_rotation * pointSet_to_rotate

                latBirth = latlonBirth.get_latitude()
                lonBirth = latlonBirth.get_longitude()

                #-------------------#
                #Set the points for coregistering
                region=5.0 #degrees

                #-------------------#
                #Coregisterring raster 1
                #-------------------#
                #Find the region in index units
                r=numpy.round(region/(x[1]-x[0]))

                #Find the index unit of lat and lon
                idxLon = (numpy.abs(x-lonBirth)).argmin()
                idxLat = (numpy.abs(y-latBirth)).argmin()

                #Raster 1
                c2=coregRaster([idxLon,idxLat],z,r)
                #Hack to search further around the age grid if it can't find a match, \
                #note index units (not degrees)
                if numpy.isnan(c2):
                    c2=coregRaster([idxLon,idxLat],z,r+150.0)
                    print("Trying raster region: ", r+150.0)

                #-------------------#
                #Coregisterring vector 1
                #-------------------#
                index=coregPoint([lonBirth,latBirth],lonlat,region)
                if index=='inf':
                    print("trying index region", region+15)
                    index=coregPoint([lonBirth,latBirth],lonlat,region+15.0)

                if numpy.isnan(c2) or index=='inf':
                    #if we have some null data, let's save it anyway, see what happens
                    print("Skipping:", i, age, t, time, c2, index, lonBirth,latBirth)

                else:
                    #Vector 1
                    segmentLength=f[index,3]
                    slabLength=f[index,9]
                    distSlabEdge=f[index,15]

                    SPcoregNor=f[index,4]
                    SPcoregPar=f[index,12]
                    OPcoregNor=f[index,5]
                    OPcoregPar=f[index,13]
                    CONVcoregNor=f[index,10]
                    CONVcoregPar=f[index,11]

                    subPolCoreg=f[index,8]
                    subOblCoreg=f[index,7]


                    coregData[i,t,:]=[shapeArray[0],shapeArray[1],lonBirth,latBirth,\
                                    age,t,c2,\
                                    segmentLength,slabLength,distSlabEdge,\
                                    SPcoregNor,SPcoregPar,OPcoregNor,OPcoregPar,CONVcoregNor,CONVcoregPar,\
                                    subPolCoreg,subOblCoreg]


    return(coregData)

In [10]:
#Create a list of points that we can track through history
#In this case, we will use, the latlon points of the SAM trench.
#We can pull out this list from a kinematic file
f = numpy.loadtxt("Muller_convergence/subStats_0.csv", delimiter=',')

In [11]:
#First, find SAM plate ID 201
lonlat=f[(f[:,17])==201]
#Then just pull out the trench points - done manually by plotting
pointlist=lonlat[379:,0:2]

In [12]:
#Check the points on a plot if you want
#plt.plot(pointlist[:,0],pointlist[:,1])

In [13]:
#Now run the coregistration
tic=time.time()
a=coregLoopHistory(pointlist,ts0=0,ts1=230,plateID=201)
toc=time.time()
print("Time taken:", toc-tic, " seconds")

/opt/conda/envs/py2GEOL/lib/python2.7/site-packages/scipy/io/netcdf.py:317: RuntimeWarning: Cannot close a netcdf_file opened with mmap=True, when netcdf_variables or arrays referring to its data still exist. All data arrays obtained from such files refer directly to data on disk, and must be copied before the file can be cleanly closed. (See netcdf_file docstring for more information on mmap.)
  ), category=RuntimeWarning)


(0, 0, 0, 0, 201)
(1, 0, 0, 0, 201)
(2, 0, 0, 0, 201)
(3, 0, 0, 0, 201)
(4, 0, 0, 0, 201)
(5, 0, 0, 0, 201)
(6, 0, 0, 0, 201)
(7, 0, 0, 0, 201)
(8, 0, 0, 0, 201)
(9, 0, 0, 0, 201)
(10, 0, 0, 0, 201)
(11, 0, 0, 0, 201)
(12, 0, 0, 0, 201)
(13, 0, 0, 0, 201)
(14, 0, 0, 0, 201)
(15, 0, 0, 0, 201)
(16, 0, 0, 0, 201)
(17, 0, 0, 0, 201)
(18, 0, 0, 0, 201)
(19, 0, 0, 0, 201)
(20, 0, 0, 0, 201)
(21, 0, 0, 0, 201)
(22, 0, 0, 0, 201)
(23, 0, 0, 0, 201)
(24, 0, 0, 0, 201)
(25, 0, 0, 0, 201)
(26, 0, 0, 0, 201)
(27, 0, 0, 0, 201)
(28, 0, 0, 0, 201)
(29, 0, 0, 0, 201)
(30, 0, 0, 0, 201)
(31, 0, 0, 0, 201)
(32, 0, 0, 0, 201)
(33, 0, 0, 0, 201)
(34, 0, 0, 0, 201)
(35, 0, 0, 0, 201)
(36, 0, 0, 0, 201)
(37, 0, 0, 0, 201)
(38, 0, 0, 0, 201)
(39, 0, 0, 0, 201)
(40, 0, 0, 0, 201)
(41, 0, 0, 0, 201)
(42, 0, 0, 0, 201)
(43, 0, 0, 0, 201)
(44, 0, 0, 0, 201)
(45, 0, 0, 0, 201)
(46, 0, 0, 0, 201)
(47, 0, 0, 0, 201)
(48, 0, 0, 0, 201)
(49, 0, 0, 0, 201)
(50, 0, 0, 0, 201)
(51, 0, 0, 0, 201)
(52, 0, 0, 0, 201)
(53

(108, 0, 1, 1, 201)
(109, 0, 1, 1, 201)
(110, 0, 1, 1, 201)
(111, 0, 1, 1, 201)
(112, 0, 1, 1, 201)
(113, 0, 1, 1, 201)
(114, 0, 1, 1, 201)
(115, 0, 1, 1, 201)
(116, 0, 1, 1, 201)
(117, 0, 1, 1, 201)
(118, 0, 1, 1, 201)
(119, 0, 1, 1, 201)
(120, 0, 1, 1, 201)
(121, 0, 1, 1, 201)
(122, 0, 1, 1, 201)
(123, 0, 1, 1, 201)
(124, 0, 1, 1, 201)
(125, 0, 1, 1, 201)
(126, 0, 1, 1, 201)
(127, 0, 1, 1, 201)
(128, 0, 1, 1, 201)
(129, 0, 1, 1, 201)
(130, 0, 1, 1, 201)
(131, 0, 1, 1, 201)
(132, 0, 1, 1, 201)
(133, 0, 1, 1, 201)
(134, 0, 1, 1, 201)
(135, 0, 1, 1, 201)
(136, 0, 1, 1, 201)
(137, 0, 1, 1, 201)
(138, 0, 1, 1, 201)
(139, 0, 1, 1, 201)
(140, 0, 1, 1, 201)
(141, 0, 1, 1, 201)
(142, 0, 1, 1, 201)
(143, 0, 1, 1, 201)
(144, 0, 1, 1, 201)
(145, 0, 1, 1, 201)
(146, 0, 1, 1, 201)
(147, 0, 1, 1, 201)
(148, 0, 1, 1, 201)
(149, 0, 1, 1, 201)
(150, 0, 1, 1, 201)
(151, 0, 1, 1, 201)
(152, 0, 1, 1, 201)
(153, 0, 1, 1, 201)
(154, 0, 1, 1, 201)
(155, 0, 1, 1, 201)
(156, 0, 1, 1, 201)
(157, 0, 1, 1, 201)


(222, 0, 2, 2, 201)
(223, 0, 2, 2, 201)
(224, 0, 2, 2, 201)
(225, 0, 2, 2, 201)
(226, 0, 2, 2, 201)
(227, 0, 2, 2, 201)
(228, 0, 2, 2, 201)
(229, 0, 2, 2, 201)
(230, 0, 2, 2, 201)
(231, 0, 2, 2, 201)
(232, 0, 2, 2, 201)
(233, 0, 2, 2, 201)
(234, 0, 2, 2, 201)
(235, 0, 2, 2, 201)
(236, 0, 2, 2, 201)
(237, 0, 2, 2, 201)
(238, 0, 2, 2, 201)
(239, 0, 2, 2, 201)
(240, 0, 2, 2, 201)
(241, 0, 2, 2, 201)
(242, 0, 2, 2, 201)
(243, 0, 2, 2, 201)
(244, 0, 2, 2, 201)
(245, 0, 2, 2, 201)
(246, 0, 2, 2, 201)
(247, 0, 2, 2, 201)
(248, 0, 2, 2, 201)
(249, 0, 2, 2, 201)
(250, 0, 2, 2, 201)
(251, 0, 2, 2, 201)
(252, 0, 2, 2, 201)
(253, 0, 2, 2, 201)
(254, 0, 2, 2, 201)
(255, 0, 2, 2, 201)
(256, 0, 2, 2, 201)
(257, 0, 2, 2, 201)
(258, 0, 2, 2, 201)
(259, 0, 2, 2, 201)
(260, 0, 2, 2, 201)
(261, 0, 2, 2, 201)
(262, 0, 2, 2, 201)
(263, 0, 2, 2, 201)
(264, 0, 2, 2, 201)
(265, 0, 2, 2, 201)
(266, 0, 2, 2, 201)
(267, 0, 2, 2, 201)
(268, 0, 2, 2, 201)
(269, 0, 2, 2, 201)
(270, 0, 2, 2, 201)
(271, 0, 2, 2, 201)


(14, 0, 4, 4, 201)
(15, 0, 4, 4, 201)
(16, 0, 4, 4, 201)
(17, 0, 4, 4, 201)
(18, 0, 4, 4, 201)
(19, 0, 4, 4, 201)
(20, 0, 4, 4, 201)
(21, 0, 4, 4, 201)
(22, 0, 4, 4, 201)
(23, 0, 4, 4, 201)
(24, 0, 4, 4, 201)
(25, 0, 4, 4, 201)
(26, 0, 4, 4, 201)
(27, 0, 4, 4, 201)
(28, 0, 4, 4, 201)
(29, 0, 4, 4, 201)
(30, 0, 4, 4, 201)
(31, 0, 4, 4, 201)
(32, 0, 4, 4, 201)
(33, 0, 4, 4, 201)
(34, 0, 4, 4, 201)
(35, 0, 4, 4, 201)
(36, 0, 4, 4, 201)
(37, 0, 4, 4, 201)
(38, 0, 4, 4, 201)
(39, 0, 4, 4, 201)
(40, 0, 4, 4, 201)
(41, 0, 4, 4, 201)
(42, 0, 4, 4, 201)
(43, 0, 4, 4, 201)
(44, 0, 4, 4, 201)
(45, 0, 4, 4, 201)
(46, 0, 4, 4, 201)
(47, 0, 4, 4, 201)
(48, 0, 4, 4, 201)
(49, 0, 4, 4, 201)
(50, 0, 4, 4, 201)
(51, 0, 4, 4, 201)
(52, 0, 4, 4, 201)
(53, 0, 4, 4, 201)
(54, 0, 4, 4, 201)
(55, 0, 4, 4, 201)
(56, 0, 4, 4, 201)
(57, 0, 4, 4, 201)
(58, 0, 4, 4, 201)
(59, 0, 4, 4, 201)
(60, 0, 4, 4, 201)
(61, 0, 4, 4, 201)
(62, 0, 4, 4, 201)
(63, 0, 4, 4, 201)
(64, 0, 4, 4, 201)
(65, 0, 4, 4, 201)
(66, 0, 4, 4

...

('trying index region', 20.0)
(10, 0, 226, 226, 201)
('trying index region', 20.0)
(11, 0, 226, 226, 201)
('trying index region', 20.0)
(12, 0, 226, 226, 201)
('trying index region', 20.0)
(13, 0, 226, 226, 201)
('trying index region', 20.0)
(14, 0, 226, 226, 201)
('trying index region', 20.0)
(15, 0, 226, 226, 201)
('trying index region', 20.0)
(16, 0, 226, 226, 201)
('trying index region', 20.0)
(17, 0, 226, 226, 201)
('trying index region', 20.0)
(18, 0, 226, 226, 201)
('trying index region', 20.0)
(19, 0, 226, 226, 201)
('trying index region', 20.0)
(20, 0, 226, 226, 201)
('trying index region', 20.0)
(21, 0, 226, 226, 201)
('trying index region', 20.0)
(22, 0, 226, 226, 201)
('trying index region', 20.0)
(23, 0, 226, 226, 201)
('trying index region', 20.0)
(24, 0, 226, 226, 201)
('trying index region', 20.0)
(25, 0, 226, 226, 201)
('trying index region', 20.0)
(26, 0, 226, 226, 201)
(27, 0, 226, 226, 201)
(28, 0, 226, 226, 201)
(29, 0, 226, 226, 201)
(30, 0, 226, 226, 201)
(31, 0,

(305, 0, 226, 226, 201)
('Trying raster region: ', 200.0)
(306, 0, 226, 226, 201)
('Trying raster region: ', 200.0)
(307, 0, 226, 226, 201)
('Trying raster region: ', 200.0)
(308, 0, 226, 226, 201)
('Trying raster region: ', 200.0)
(309, 0, 226, 226, 201)
('Trying raster region: ', 200.0)
(310, 0, 226, 226, 201)
('Trying raster region: ', 200.0)
(311, 0, 226, 226, 201)
('Trying raster region: ', 200.0)
(312, 0, 226, 226, 201)
('Trying raster region: ', 200.0)
(0, 0, 227, 227, 201)
('trying index region', 20.0)
(1, 0, 227, 227, 201)
('trying index region', 20.0)
(2, 0, 227, 227, 201)
('trying index region', 20.0)
(3, 0, 227, 227, 201)
('trying index region', 20.0)
(4, 0, 227, 227, 201)
('trying index region', 20.0)
(5, 0, 227, 227, 201)
('trying index region', 20.0)
(6, 0, 227, 227, 201)
('trying index region', 20.0)
(7, 0, 227, 227, 201)
('trying index region', 20.0)
(8, 0, 227, 227, 201)
('trying index region', 20.0)
(9, 0, 227, 227, 201)
('trying index region', 20.0)
(10, 0, 227, 227

('Trying raster region: ', 200.0)
(291, 0, 227, 227, 201)
('Trying raster region: ', 200.0)
(292, 0, 227, 227, 201)
('Trying raster region: ', 200.0)
(293, 0, 227, 227, 201)
('Trying raster region: ', 200.0)
(294, 0, 227, 227, 201)
('Trying raster region: ', 200.0)
(295, 0, 227, 227, 201)
('Trying raster region: ', 200.0)
(296, 0, 227, 227, 201)
('Trying raster region: ', 200.0)
(297, 0, 227, 227, 201)
('Trying raster region: ', 200.0)
(298, 0, 227, 227, 201)
('Trying raster region: ', 200.0)
(299, 0, 227, 227, 201)
('Trying raster region: ', 200.0)
(300, 0, 227, 227, 201)
('Trying raster region: ', 200.0)
(301, 0, 227, 227, 201)
('Trying raster region: ', 200.0)
(302, 0, 227, 227, 201)
('Trying raster region: ', 200.0)
(303, 0, 227, 227, 201)
('Trying raster region: ', 200.0)
(304, 0, 227, 227, 201)
('Trying raster region: ', 200.0)
(305, 0, 227, 227, 201)
('Trying raster region: ', 200.0)
(306, 0, 227, 227, 201)
('Trying raster region: ', 200.0)
(307, 0, 227, 227, 201)
('Trying raste

(252, 0, 228, 228, 201)
(253, 0, 228, 228, 201)
(254, 0, 228, 228, 201)
(255, 0, 228, 228, 201)
(256, 0, 228, 228, 201)
(257, 0, 228, 228, 201)
(258, 0, 228, 228, 201)
(259, 0, 228, 228, 201)
(260, 0, 228, 228, 201)
(261, 0, 228, 228, 201)
(262, 0, 228, 228, 201)
(263, 0, 228, 228, 201)
(264, 0, 228, 228, 201)
(265, 0, 228, 228, 201)
(266, 0, 228, 228, 201)
(267, 0, 228, 228, 201)
(268, 0, 228, 228, 201)
(269, 0, 228, 228, 201)
(270, 0, 228, 228, 201)
(271, 0, 228, 228, 201)
(272, 0, 228, 228, 201)
(273, 0, 228, 228, 201)
(274, 0, 228, 228, 201)
(275, 0, 228, 228, 201)
(276, 0, 228, 228, 201)
(277, 0, 228, 228, 201)
(278, 0, 228, 228, 201)
(279, 0, 228, 228, 201)
(280, 0, 228, 228, 201)
(281, 0, 228, 228, 201)
(282, 0, 228, 228, 201)
(283, 0, 228, 228, 201)
(284, 0, 228, 228, 201)
(285, 0, 228, 228, 201)
(286, 0, 228, 228, 201)
(287, 0, 228, 228, 201)
(288, 0, 228, 228, 201)
(289, 0, 228, 228, 201)
(290, 0, 228, 228, 201)
('Trying raster region: ', 200.0)
(291, 0, 228, 228, 201)
('Tryi

(223, 0, 229, 229, 201)
(224, 0, 229, 229, 201)
(225, 0, 229, 229, 201)
(226, 0, 229, 229, 201)
(227, 0, 229, 229, 201)
(228, 0, 229, 229, 201)
(229, 0, 229, 229, 201)
(230, 0, 229, 229, 201)
(231, 0, 229, 229, 201)
(232, 0, 229, 229, 201)
(233, 0, 229, 229, 201)
(234, 0, 229, 229, 201)
(235, 0, 229, 229, 201)
(236, 0, 229, 229, 201)
(237, 0, 229, 229, 201)
(238, 0, 229, 229, 201)
(239, 0, 229, 229, 201)
(240, 0, 229, 229, 201)
(241, 0, 229, 229, 201)
(242, 0, 229, 229, 201)
(243, 0, 229, 229, 201)
(244, 0, 229, 229, 201)
(245, 0, 229, 229, 201)
(246, 0, 229, 229, 201)
(247, 0, 229, 229, 201)
(248, 0, 229, 229, 201)
(249, 0, 229, 229, 201)
(250, 0, 229, 229, 201)
(251, 0, 229, 229, 201)
(252, 0, 229, 229, 201)
(253, 0, 229, 229, 201)
(254, 0, 229, 229, 201)
(255, 0, 229, 229, 201)
(256, 0, 229, 229, 201)
(257, 0, 229, 229, 201)
(258, 0, 229, 229, 201)
(259, 0, 229, 229, 201)
(260, 0, 229, 229, 201)
(261, 0, 229, 229, 201)
(262, 0, 229, 229, 201)
(263, 0, 229, 229, 201)
(264, 0, 229, 22

In [14]:
#And save out the dataset
andesPresent=numpy.array(a)
andeanOutSamp=open(r'Muller_Bertrand_coregistered_sampleMuller0.pkl','wb')
pickle.dump(andesPresent,andeanOutSamp)
andeanOutSamp.close()

In [5]:
#Now run the coregestration on the ore deposit data, to find the 
#Tectonmagmatic properties around the time a deposit was formed.

#Load in the data (in shapefile format)
andeanPoints="CopperDeposits/XYBer14_t2_ANDES.shp"
print andeanPoints
[recs,shapes,fields,Nshp]=readTopologyPlatepolygonFile(andeanPoints)

CopperDeposits/XYBer14_t2_ANDES.shp


In [6]:
#Wrangle the data into a format we can use
sampleData=numpy.zeros((Nshp,4))

for i in xrange(Nshp):
    sampleData[i,2]=recs[i][6] #Ages
    sampleData[i,3]=recs[i][7] #PlateIDs
    sampleData[i,0]=recs[i][3] #Longs
    sampleData[i,1]=recs[i][4] #Lats

In [17]:
#Run the coregistration
tic=time.time()
b=coregLoop(sampleData[:,0:2],sampleData[:,2],sampleData[:,3])
toc=time.time()
print("Time taken:", toc-tic, " seconds")

('Deposit:', 0, 'of', 155, 'Lat:', -27.37, 'Lon:', -66.28, 'Age:', 5.5, 'PlateID:', 201)
('Trying raster region: ', 200.0)
('trying index region', 20.0)
('Trying raster region: ', 200.0)
('trying index region', 20.0)
('Deposit:', 1, 'of', 155, 'Lat:', -31.3, 'Lon:', -69.37, 'Age:', 267.0, 'PlateID:', 201)
('Deposit:', 2, 'of', 155, 'Lat:', -30.5, 'Lon:', -69.75, 'Age:', 12.0, 'PlateID:', 201)
('Deposit:', 3, 'of', 155, 'Lat:', -27.27, 'Lon:', -66.65, 'Age:', 8.5, 'PlateID:', 201)
('Deposit:', 4, 'of', 155, 'Lat:', -27.33, 'Lon:', -66.61, 'Age:', 7.5, 'PlateID:', 201)
('Deposit:', 5, 'of', 155, 'Lat:', -27.4, 'Lon:', -66.55, 'Age:', 7.0, 'PlateID:', 201)
('Trying raster region: ', 200.0)
('trying index region', 20.0)
('Deposit:', 6, 'of', 155, 'Lat:', -27.28, 'Lon:', -66.57, 'Age:', 8.0, 'PlateID:', 201)
('trying index region', 20.0)
('Deposit:', 7, 'of', 155, 'Lat:', -26.3, 'Lon:', -67.9, 'Age:', 14.0, 'PlateID:', 201)
('Deposit:', 8, 'of', 155, 'Lat:', -38.25, 'Lon:', -70.58, 'Age:', 

('Deposit:', 71, 'of', 155, 'Lat:', -19.81, 'Lon:', -69.28, 'Age:', 58.0, 'PlateID:', 201)
('Trying raster region: ', 200.0)
('Deposit:', 72, 'of', 155, 'Lat:', -22.47, 'Lon:', -68.97, 'Age:', 35.5, 'PlateID:', 201)
('Deposit:', 73, 'of', 155, 'Lat:', -23.3, 'Lon:', -69.12, 'Age:', 40.0, 'PlateID:', 201)
('Deposit:', 74, 'of', 155, 'Lat:', -26.49, 'Lon:', -69.42, 'Age:', 36.5, 'PlateID:', 201)
('Deposit:', 75, 'of', 155, 'Lat:', -27.48, 'Lon:', -70.25, 'Age:', 109.6, 'PlateID:', 201)
('Deposit:', 76, 'of', 155, 'Lat:', -21.92, 'Lon:', -69.83, 'Age:', 132.0, 'PlateID:', 201)
('Trying raster region: ', 200.0)
('Deposit:', 77, 'of', 155, 'Lat:', -21.0, 'Lon:', -68.8, 'Age:', 36.0, 'PlateID:', 201)
('Deposit:', 78, 'of', 155, 'Lat:', -19.87, 'Lon:', -68.97, 'Age:', 36.0, 'PlateID:', 201)
('Deposit:', 79, 'of', 155, 'Lat:', -27.38, 'Lon:', -69.27, 'Age:', 22.8, 'PlateID:', 201)
('Deposit:', 80, 'of', 155, 'Lat:', -28.5, 'Lon:', -70.3, 'Age:', 64.0, 'PlateID:', 201)
('Deposit:', 81, 'of', 15

('Trying raster region: ', 200.0)
('Deposit:', 154, 'of', 155, 'Lat:', -11.6, 'Lon:', -76.13, 'Age:', 7.5, 'PlateID:', 201)
('Time taken:', 43.957550048828125, ' seconds')


In [18]:
#Save the data out
andes=numpy.array(b)
andeanOut=open(r'Muller_Bertrand_coregistered.pkl','wb')
pickle.dump(andes,andeanOut)
andeanOut.close()

In [7]:
#Now make a set of "non-deposits" using a random formation age
randomAges=numpy.random.randint(1,230,size=len(sampleData))

In [8]:
tic=time.time()
c=coregLoop(sampleData[:,0:2],randomAges,sampleData[:,3])
toc=time.time()
print("Time taken:", toc-tic, " seconds")

('Deposit:', 0, 'of', 155, 'Lat:', -27.37, 'Lon:', -66.28, 'Age:', 109, 'PlateID:', 201)


/opt/conda/envs/py2GEOL/lib/python2.7/site-packages/scipy/io/netcdf.py:317: RuntimeWarning: Cannot close a netcdf_file opened with mmap=True, when netcdf_variables or arrays referring to its data still exist. All data arrays obtained from such files refer directly to data on disk, and must be copied before the file can be cleanly closed. (See netcdf_file docstring for more information on mmap.)
  ), category=RuntimeWarning)
Utils_coreg.py:165: RuntimeWarning: Mean of empty slice
  return(numpy.nanmean(pts))


('Trying raster region: ', 200.0)
('trying index region', 20.0)
('Deposit:', 1, 'of', 155, 'Lat:', -31.3, 'Lon:', -69.37, 'Age:', 34, 'PlateID:', 201)
('Deposit:', 2, 'of', 155, 'Lat:', -30.5, 'Lon:', -69.75, 'Age:', 181, 'PlateID:', 201)
('Deposit:', 3, 'of', 155, 'Lat:', -27.27, 'Lon:', -66.65, 'Age:', 207, 'PlateID:', 201)
('Trying raster region: ', 200.0)
('trying index region', 20.0)
('Deposit:', 4, 'of', 155, 'Lat:', -27.33, 'Lon:', -66.61, 'Age:', 101, 'PlateID:', 201)
('Trying raster region: ', 200.0)
('trying index region', 20.0)
('Deposit:', 5, 'of', 155, 'Lat:', -27.4, 'Lon:', -66.55, 'Age:', 154, 'PlateID:', 201)
('Trying raster region: ', 200.0)
('trying index region', 20.0)
('Deposit:', 6, 'of', 155, 'Lat:', -27.28, 'Lon:', -66.57, 'Age:', 39, 'PlateID:', 201)
('Trying raster region: ', 200.0)
('trying index region', 20.0)
('Deposit:', 7, 'of', 155, 'Lat:', -26.3, 'Lon:', -67.9, 'Age:', 161, 'PlateID:', 201)
('Trying raster region: ', 200.0)
('Deposit:', 8, 'of', 155, 'La

('Trying raster region: ', 200.0)
('Deposit:', 72, 'of', 155, 'Lat:', -22.47, 'Lon:', -68.97, 'Age:', 132, 'PlateID:', 201)
('Trying raster region: ', 200.0)
('Deposit:', 73, 'of', 155, 'Lat:', -23.3, 'Lon:', -69.12, 'Age:', 72, 'PlateID:', 201)
('Trying raster region: ', 200.0)
('Deposit:', 74, 'of', 155, 'Lat:', -26.49, 'Lon:', -69.42, 'Age:', 22, 'PlateID:', 201)
('Deposit:', 75, 'of', 155, 'Lat:', -27.48, 'Lon:', -70.25, 'Age:', 183, 'PlateID:', 201)
('Deposit:', 76, 'of', 155, 'Lat:', -21.92, 'Lon:', -69.83, 'Age:', 147, 'PlateID:', 201)
('Trying raster region: ', 200.0)
('Deposit:', 77, 'of', 155, 'Lat:', -21.0, 'Lon:', -68.8, 'Age:', 158, 'PlateID:', 201)
('Trying raster region: ', 200.0)
('Deposit:', 78, 'of', 155, 'Lat:', -19.87, 'Lon:', -68.97, 'Age:', 106, 'PlateID:', 201)
('Trying raster region: ', 200.0)
('Deposit:', 79, 'of', 155, 'Lat:', -27.38, 'Lon:', -69.27, 'Age:', 222, 'PlateID:', 201)
('Trying raster region: ', 200.0)
('Deposit:', 80, 'of', 155, 'Lat:', -28.5, 'Lon

('Deposit:', 151, 'of', 155, 'Lat:', -13.69, 'Lon:', -75.09, 'Age:', 166, 'PlateID:', 201)
('Deposit:', 152, 'of', 155, 'Lat:', -14.91, 'Lon:', -71.31, 'Age:', 226, 'PlateID:', 201)
('Trying raster region: ', 200.0)
('Deposit:', 153, 'of', 155, 'Lat:', -17.25, 'Lon:', -70.61, 'Age:', 160, 'PlateID:', 201)
('Trying raster region: ', 200.0)
('Deposit:', 154, 'of', 155, 'Lat:', -11.6, 'Lon:', -76.13, 'Age:', 7, 'PlateID:', 201)
('Time taken:', 47.78632307052612, ' seconds')


In [9]:
#Save the random coregistered data out
andesRand=numpy.array(c)
andeanOutRand=open(r'Muller_Bertrand_coregistered_random.pkl','wb')
pickle.dump(andesRand,andeanOutRand)
andeanOutRand.close()